In this tutorial we will see what it takes to load and render an .obj model using the pyGanadalf API.
-----------------------------------------------------------------------------------------------------

Before doing anything else we need to import the nessasary packages

In [1]:
from pyGandalf.core.application import Application
from pyGandalf.core.opengl_window import OpenGLWindow
from pyGandalf.systems.transform_system import TransformSystem
from pyGandalf.systems.link_system import LinkSystem
from pyGandalf.systems.camera_system import CameraSystem
from pyGandalf.systems.opengl_rendering_system import OpenGLStaticMeshRenderingSystem

from pyGandalf.renderer.opengl_renderer import OpenGLRenderer

from pyGandalf.scene.scene import Scene
from pyGandalf.scene.components import *
from pyGandalf.scene.scene_manager import SceneManager

from pyGandalf.utilities.opengl_material_lib import OpenGLMaterialLib, MaterialData
from pyGandalf.utilities.opengl_texture_lib import OpenGLTextureLib, TextureData
from pyGandalf.utilities.opengl_shader_lib import OpenGLShaderLib
from pyGandalf.utilities.mesh_lib import MeshLib

from pyGandalf.utilities.definitions import SHADERS_PATH, MODELS_PATH

import numpy as np
import glm

First, we create a new application. To achieve this, we need to specify two main things:
- The window
  - We create an `OpenGLWindow` and we specify the name, width, height and vertical sync mode accordingly.
- The renderer
  - We specify that we will use the `OpenGLRenderer`.

In [2]:
Application().create(OpenGLWindow('Scene Graph', 1280, 720, True), OpenGLRenderer)

Then, we build a shader, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLShaderLib` we call the `build` method to build our shader.
- 2. We give our shader a name, in this case we chose: "unlit".
- 3. We provide the build function with the path to the source code of the shader stages that we want our shader to utilize, in this case we use a vertext and fragment shader.
    
    In this example we create a simple shader that implements the Blinn-Phong shading method to shade the model that we will load.

In [3]:
OpenGLShaderLib().build('unlit', SHADERS_PATH / 'opengl' / 'unlit_simple.vs', SHADERS_PATH / 'opengl' / 'unlit_simple.fs')

3

Then, we build a material, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLMaterialLib` we call the `build` method to build our material.
- 2. We give the material a name, in this case we chose: "M_Unlit".
- 3. We give specify the desired data of the material through the `MaterialData` class.
  - As the first argument, we give them shader name that we want to use, we give the name of the shader we created above, meaning: "lit_blinn_phong".
  - The second argument is used to define which textures the material we use, in our case we dont have textures so we leave the array empty.

In [4]:
OpenGLMaterialLib().build('M_Unlit', MaterialData('unlit', []))

We create a new scene by instatiating an object of the class `Scene`, we give it the name "Scene Graph".

In [5]:
scene = Scene('Scene Graph')

Then, we enroll a new entity to the scene by calling the `enroll_entity` method. This entity will represent the root of our scene hierachy.

In [6]:
root = scene.enroll_entity()

Now, its time to add components to our newly created entity, i.e.: `root`.

First, using our `scene` and its `add_component` method, we add to the `root` entity, a `TransformComponent`.

We when are creating the `TransformComponent`, we specify the translation, rotation and scale vectors.

In [7]:
scene.add_component(root, TransformComponent(glm.vec3(0, 0, 0), glm.vec3(0, 0, 0), glm.vec3(1, 1, 1)))

Then, we want to add a `LinkComponent` component to the `root` entity.
- We add the `LinkComponent` specifying the entity that we want to be this entity's parent.
- In this case, this is the root so we pass to it `None`.

In [8]:
scene.add_component(root, LinkComponent(None))

Then, we enroll a new entity to the scene by calling the `enroll_entity` method. This entity will a child of our `root` entity.

In [9]:
child = scene.enroll_entity()

Now, using our `scene` and its `add_component` method, we add to the `child` entity, a `TransformComponent`.

We when are creating the `TransformComponent`, we specify the translation, rotation and scale vectors.
- We translate it 2 units on the x-axis to the left.

In [10]:
scene.add_component(child, TransformComponent(glm.vec3(-2, 0, 0), glm.vec3(0, 0, 0), glm.vec3(1, 1, 1)))

Then, we want to add a `LinkComponent` component to the `child` entity.
- We add the `LinkComponent` specifying the entity that we want to be this entity's parent.
- In this case, the parent is the `root` entity that we created above.

In [11]:
scene.add_component(child, LinkComponent(root))

In order to see our child entity rendered, we need to add a `StaticMeshComponent` to it specifying a name, we chose "quad" in this case.

Then, we specify in an array all the vertex data, in our case we have the vertices as shown below.

In [12]:
vertices = np.array([
    [-0.5, -0.5, 0.0], # 0 - Bottom left corner
    [ 0.5, -0.5, 0.0], # 1 - Bottom right corner
    [ 0.5,  0.5, 0.0], # 2 - Top right corner
    [ 0.5,  0.5, 0.0], # 2 - Top right corner
    [-0.5,  0.5, 0.0], # 3 - Top left corner
    [-0.5, -0.5, 0.0]  # 0 - Bottom left corner
], dtype=np.float32)

In [13]:
scene.add_component(child, StaticMeshComponent('quad', [vertices]))

Next, we have to add the `MaterialComponent`.

We just have to give the name of the material we have built before, i.e.: "M_Unlit".

In [14]:
scene.add_component(child, MaterialComponent('M_Unlit'))

Now, lets create a new entity in our scene, using the `enroll_entity` method, for the camera.

In [15]:
camera = scene.enroll_entity()

Then, using our `scene` and its `add_component` method, we add to the `camera` entity, a `TransformComponent`.

We position the camera 5 meters back.

In [16]:
scene.add_component(camera, TransformComponent(glm.vec3(0, 0, 5), glm.vec3(0, 0, 0), glm.vec3(1, 1, 1)))

Then, using our `scene` and its `add_component` method, we add to the `camera` entity, a `CameraComponent`.

We provide the `CameraComponent` with the required data: field of view, aspect ratio, near plane, far plane and the type `CameraComponent.Type.PERSPECTIVE` or `CameraComponent.Type.ORTHOGRAPHIC`

In [17]:
scene.add_component(camera, CameraComponent(45, 1.778, 0.1, 1000, 1.2, CameraComponent.Type.PERSPECTIVE))

Now that we have the components added to the entities, we have to register systems that we are gonna use to the scene.
In this example we are using two systems:
- `TransformSystem`
- `LinkSystem`
- `CameraSystem`
- `OpenGLStaticMeshRenderingSystem`

When creating a system we have to provide to the constructor an array that holds the component(s) that the system operates on.

Then the system uses these component(s) to filter all the entities that have them.
- For example, the `TransformSystem` operates on all the entities that have a `TransformComponent`.
  - The `LinkSystem` operates on all the entities that have a `LinkComponent` and a `TransformComponent`.
  - Similarly the `OpenGLStaticMeshRenderingSystem` operates on all the entities that have a `StaticMeshComponent`, a `MaterialComponent` and a `TransformComponent`.
  - Finally, the `CameraSystem` operates on all the entities that have a `CameraComponent` and a `TransformComponent`.

We can register them to the scene by calling the `register_system` method.

In [18]:
scene.register_system(TransformSystem([TransformComponent]))
scene.register_system(LinkSystem([LinkComponent, TransformComponent]))
scene.register_system(CameraSystem([CameraComponent, TransformComponent]))
scene.register_system(OpenGLStaticMeshRenderingSystem([StaticMeshComponent, MaterialComponent, TransformComponent]))

Now that our scene has all the entites, components and systems added we can add it to the `SceneManager` by calling the `add_scene` method.

In [19]:
SceneManager().add_scene(scene)

Finally, we can start the `Application` by calling the `start` method.

In [20]:
Application().start()